In [ ]:
from pyelasticsearch import ElasticSearch
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, MetaData, Table, select
from time import time

servername = "localhost\sqlexpress"
database = "Tableau"
serverdriver = "sql server"



In [ ]:
es = ElasticSearch('http://localhost:9200/')

index = "latest_aging"
doc = "latest_entry"

settings ={
    'test':{
    'properties':{
    'companyname':{'index':'not_analyzed','type': 'string'},
    'diagnosisdescription':{'index':'not_analyzed', 'type':'string'}b
         }
    }
}

In [ ]:
engine = create_engine('mssql+pyodbc://'+servername+'/'+database+'?driver='+serverdriver)

date = ''' '20160115' '''


chunk = pd.read_sql('select top 5 PolicyNo,ClientName,[Inter Type],[Inter Name] \
                    ,AM,[Bill Description],Age,[Age Bracket],recordDate  \
                    ,[Bill/Cheque No] ,[Gross O/S],[Curr Annual Prem] \
                    from [dbo].[G400AgingClosing] where cast(recorddate as date)=' + date,engine,chunksize = 1)




settings ={
    'records':{
        'properties':{
            'PolicyNo':{'index':'not_analyzed','type':'string'},
            'ClientName':{'index':'not_analyzed','type':'string'},
            'Inter Type':{'index':'not_analyzed','type':'string'},
            'Inter Name':{'index':'not_analyzed','type':'string'},
            'AM':{'index':'not_analyzed','type':'string'},
            'Bill Description':{'type':'string'},
            'Age':{'type':'integer'},
            'Age Bracket':{'index':'not_analyzed','type':'string'},
            'Bill/Cheque No':{'index':'not_analyzed','type':'string'},
            'Gross O/S':{'index':'not_analyzed','type':'float'},
            'Curr Annual Prem':{'index':'not_analyzed','type':'integer'}
            
        }
    }
}





In [ ]:
try :
    es.delete_index(index)
except :
    pass

es.create_index(index, settings={'mappings': settings})

In [ ]:
t0 = time()

print "now indexing %s..."%(database)

for i,df in enumerate(chunk): 
    print i
    records=df.where(pd.notnull(df), None).T.to_dict()
    list_records=[records[it] for it in records]
    try :
        #es.bulk_index(index,doc,list_records)
        print records
        print list_records
    except :
        print "error!, skiping some, sorry"
        pass

print "done in %.3fs"%(time()-t0)

In [ ]:
t0 = time()

print "now indexing %s..."%(database)

for i,df in enumerate(chunk): 
    print i
    records=df.where(pd.notnull(df), None).T.to_dict()
    list_records=[records[it] for it in records]
    try :
        es.bulk_index(index,doc,list_records)
    except :
        print "error!, skiping some, sorry"
        pass

print "done in %.3fs"%(time()-t0)

In [1]:
import numpy as np
import pandas as pd
import glob

In [ ]:
path = raw_input('Input your file path : ') # use your path
file = raw_input('File type? : ')
filetype =  "/*." + file
allFiles = glob.glob(path + filetype)
df = pd.DataFrame()
list_ = []


for file_ in allFiles:
    if filetype == "/*.csv":
        chunks = pd.read_csv(file_,index_col=None, header=0, chunksize = 10)
        for chunk in chunks:
            list_.append(chunk)
            df = pd.concat(list_, ignore_index = True)
    else:
        xls = pd.ExcelFile(file_)
        if len(xls.sheet_names) > 0:
            for sheet in xls.sheet_names:
                sd = xls.parse(sheet)
                df = df.append(sd, ignore_index = True)
        else:
            sd = pd.read_excel(file_,index_col=None, header=0)
            df = df.append(sd, ignore_index = True)

        
        

objectlist = list(df.select_dtypes(include=['object']).columns)

pd.pivot_table(df,index=objectlist)


    
